# CORAL Loss Experiments: Walkability Score Prediction

This notebook runs systematic experiments comparing different **image backbones**, **tabular encoders**, and **fusion mechanisms** using CORAL ordinal regression loss for predicting walkability scores (1-5) from street-view images and user demographics.

**Data**: ~24K training / ~6K test samples with:
- **Image**: Street-view photographs (`image_path`)
- **Tabular**: age, gender, childhood_country, childhood_area, disability, walking_frequency, residence_type
- **Label**: `rating` (ordinal 1-5)

In [ ]:
import autogluon
import autogluon.multimodal
import autogluon.tabular

print(autogluon.__path__)                   # namespace search paths
print(autogluon.multimodal.__file__)        # actual file path
print(autogluon.tabular.__file__)

_NamespacePath(['/Users/ahmademami/Library/CloudStorage/OneDrive-UNSW/TERM 1/Github/aglone/autogluon/autogluon', 'autogluon', '__editable__.autogluon-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_common-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_core-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_eda-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_features-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_multimodal-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_tabular-1.4.1b20260213.finder.__path_hook__', 'autogluon', '__editable__.autogluon_timeseries-1.4.1b20260213.finder.__path_hook__'])
/Users/ahmademami/Library/CloudStorage/OneDrive-UNSW/TERM 1/Github/aglone/autogluon/multimodal/src/autogluon/multimodal/__init__.py
/Users/ahmademami/Library/CloudStorage/OneDrive-UNSW/TERM 1/Github/aglone/autogluon/tabular/src/aut

In [ ]:
# ---------------------------
# Load and Prepare Data
# ---------------------------
np.random.seed(42)

print("Loading data...")
train_df = pd.read_csv(TRAIN_CSV, index_col=0)
test_df  = pd.read_csv(TEST_CSV,  index_col=0)

# Drop non-feature columns
train_df = train_df.drop(columns=[c for c in DROP_COLS if c in train_df.columns])
test_df  = test_df.drop(columns=[c for c in DROP_COLS if c in test_df.columns])

# Use a subset for quick CPU experiments (increase for GPU)
TRAIN_SAMPLE = 200    # ← Increase to len(train_df) for full training
TEST_SAMPLE  = 100    # ← Increase to len(test_df) for full evaluation

train_df = train_df.sample(n=min(TRAIN_SAMPLE, len(train_df)), random_state=42).reset_index(drop=True)
test_df  = test_df.sample(n=min(TEST_SAMPLE, len(test_df)), random_state=42).reset_index(drop=True)

# Absolute image paths
train_df = add_abs_image_paths(train_df, IMAGE_COL, IMG_DIR)
test_df  = add_abs_image_paths(test_df,  IMAGE_COL, IMG_DIR)

# Verify files exist
missing = (~train_df[IMAGE_COL].map(os.path.exists)).sum()
print(f"Missing image files: {missing}/{len(train_df)}")

# Data summary
print(f"\nTrain: {len(train_df)} samples, Test: {len(test_df)} samples")
print(f"Columns: {list(train_df.columns)}")
print(f"\nLabel distribution:")
print(train_df[LABEL_COL].value_counts().sort_index())
print(f"\nSample row:\n{train_df.iloc[0]}")

No path specified. Models will be saved in: "AutogluonModels/ag-20260213_055335"
=================== System Info ===================
AutoGluon Version:  1.4.1b20260213
Python Version:     3.10.19
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:08:48 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T8132
CPU Count:          10
Pytorch Version:    2.7.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       4.14 GB / 16.00 GB (25.9%)
Disk Space Avail:   314.12 GB / 460.43 GB (68.2%)


Loading data...
Using 50 training samples for quick CPU test
Missing files - Train: 0, Test: 0

Label distribution in training data:
rating
1     4
2     5
3    16
4    17
5     8
Name: count, dtype: int64

Unique classes: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

TESTING CORAL LOSS ON CPU
Loss function: coral
Epochs: 3
Batch size: 4
Training samples: 50
Test samples: 20

Starting training with CORAL loss...
⚠️ LOOK FOR DEBUG MESSAGES ABOUT LOSS FUNCTION SELECTION ⚠️




AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /Users/ahmademami/Library/CloudStorage/OneDrive-UNSW/TERM 1/Github/aglone/autogluon/AutogluonModels/ag-20260213_055335
    ```

Seed set to 0
The hyperparameter name optim.learning_rate is depreciated. We recommend using the new name optim.lr instead.The deprecated hyperparameter will raise an exception starting in AutoGluon 1.4.0
GPU Count: 1
GPU Count to be Used: 0

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.

  | Name              | Type                | Params | Mode  | FLOPs
----------------------------------------------

Output()


get_loss_func_per_run called in base.py:
  config.optim.loss_func: coral
  problem_type: multiclass
  mixup_active: False
  model type: <class 'autogluon.multimodal.models.fusion.fusion_mlp.MultimodalFusionMLP'>
  model has num_classes? True
  model.num_classes: 4


get_loss_func called:
  problem_type: multiclass
  loss_func_name: coral
  mixup_active: False
  kwargs: {'num_classes': 5}

DEBUG: Initializing CoralLoss with kwargs keys: ['num_classes']

🔥 CORAL LOSS ACTIVATED!
   num_classes: 5
   output_dim: 4 (cumulative ordinal regression)



...

Output()

💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Output()

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/Users/ahmademami/Library/CloudStorage/OneDrive-UNSW/TERM 1/Github/aglone/autogluon/AutogluonModels/ag-20260213_055335")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).





Training completed!


## Available Models in AutoGluon Multimodal

### Image Backbones (`model.timm_image.checkpoint_name`)
Any model from the [timm library](https://github.com/huggingface/pytorch-image-models) (1000+ models). Key options:

| Model | Size | Notes |
|---|---|---|
| `resnet18` | 11M | Fast baseline |
| `resnet50` | 25M | Standard CNN |
| `mobilenetv3_large_100` | 5M | Mobile-efficient (medium_quality preset) |
| `efficientnet_b0` | 5M | Efficient scaling |
| `efficientnet_b3` | 12M | Better accuracy |
| `convnext_tiny` | 28M | Modern ConvNet |
| `convnext_base_in22ft1k` | 89M | Strong ConvNet |
| `swin_base_patch4_window7_224` | 88M | Vision Transformer |
| `swin_large_patch4_window7_224` | 197M | Large ViT (best_quality preset) |
| `vit_base_patch16_clip_224.laion2b_ft_in12k_in1k` | 87M | CLIP-pretrained ViT |
| `caformer_b36.sail_in22k_ft_in1k` | 99M | MetaFormer (high_quality preset) |

### Text/Tabular Encoders (`model.hf_text.checkpoint_name`)
Any model from [HuggingFace](https://huggingface.co/models). Key options:

| Model | Size | Notes |
|---|---|---|
| `prajjwal1/bert-tiny` | 4M | Ultra-fast testing |
| `google/electra-small-discriminator` | 14M | Medium preset |
| `google/electra-base-discriminator` | 110M | High_quality preset |
| `microsoft/deberta-v3-small` | 44M | Strong text encoder |
| `microsoft/deberta-v3-base` | 86M | Best_quality preset |
| `distilroberta-base` | 82M | Fast & accurate |

### Tabular-Specific Models
| Model | Config Key | Notes |
|---|---|---|
| Numerical MLP | `model.numerical_mlp.*` | MLP for numerical features |
| Categorical MLP | `model.categorical_mlp.*` | Embedding + MLP for categories |
| FT-Transformer | `model.ft_transformer.*` | Transformer for tabular data |

### Fusion Mechanisms
| Fusion | Config Key | Notes |
|---|---|---|
| **MLP Fusion** | `model.fusion_mlp.*` | Concatenate + MLP (default, fast) |
| **Transformer Fusion** | `model.fusion_transformer.*` | Cross-attention between modalities |

In [ ]:
# ============================================================
# EXPERIMENT CONFIGURATIONS
# ============================================================
# Each experiment tests a different combination of:
#   - Loss function (CORAL vs CrossEntropy)
#   - Image backbone
#   - Tabular processing (text encoder converts tabular to text,
#     or use dedicated numerical_mlp + categorical_mlp)
#   - Fusion mechanism (MLP vs Transformer)
#
# To run on GPU: set "env.num_gpus": 1 and increase batch size
# To use full data: increase TRAIN_SAMPLE / TEST_SAMPLE above
# ============================================================

MAX_EPOCHS = 5          # ← Increase for real experiments (10-20)
TIME_LIMIT = 600        # seconds per experiment
NUM_GPUS   = 0          # ← Set to 1 for GPU training
BATCH_SIZE = 4 if NUM_GPUS == 0 else 32

# Base config shared across experiments
base_config = {
    "optim.max_epochs": MAX_EPOCHS,
    "optim.learning_rate": 1e-4,
    "env.num_gpus": NUM_GPUS,
    "env.per_gpu_batch_size": BATCH_SIZE,
    "env.num_workers": 0 if NUM_GPUS == 0 else 4,
    "env.num_workers_inference": 0 if NUM_GPUS == 0 else 4,
}

# ============================================================
# Define experiments
# ============================================================
experiments = {}

# --- 1. CORAL + ResNet18 (small baseline) ---
experiments["coral_resnet18"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet18",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 2. CrossEntropy + ResNet18 (baseline comparison) ---
experiments["ce_resnet18"] = {
    **base_config,
    # no loss_func → defaults to CrossEntropyLoss
    "model.timm_image.checkpoint_name": "resnet18",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 3. CORAL + ResNet50 (stronger CNN) ---
experiments["coral_resnet50"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet50",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 4. CORAL + EfficientNet-B0 (efficient architecture) ---
experiments["coral_efficientnet_b0"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "efficientnet_b0",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 5. CORAL + ConvNeXt-Tiny (modern ConvNet) ---
experiments["coral_convnext_tiny"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "convnext_tiny",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 6. CORAL + Swin-Base (Vision Transformer) ---
experiments["coral_swin_base"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "swin_base_patch4_window7_224",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 7. CORAL + ResNet18 + Electra-Small (better text encoder for tabular) ---
experiments["coral_resnet18_electra_small"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet18",
    "model.hf_text.checkpoint_name": "google/electra-small-discriminator",
}

# --- 8. CORAL + ResNet18 + DeBERTa-v3-small (strong text encoder) ---
experiments["coral_resnet18_deberta_small"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet18",
    "model.hf_text.checkpoint_name": "microsoft/deberta-v3-small",
}

# --- 9. CORAL + ResNet50 + Transformer Fusion ---
experiments["coral_resnet50_transformer_fusion"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet50",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
    "model.names": ["timm_image", "hf_text", "fusion_transformer"],
    "model.fusion_transformer.hidden_size": 128,
    "model.fusion_transformer.num_blocks": 2,
}

# --- 10. CORAL + ResNet18 + FT-Transformer (tabular-specific) ---
# Uses FT-Transformer for tabular features instead of text encoder
experiments["coral_resnet18_ft_transformer"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "resnet18",
    "model.names": ["timm_image", "ft_transformer", "fusion_mlp"],
    "model.ft_transformer.num_blocks": 2,
}

# --- 11. CORAL + MobileNetV3 (lightweight, fast) ---
experiments["coral_mobilenetv3"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "mobilenetv3_large_100",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

# --- 12. CORAL + ViT-Base CLIP (CLIP-pretrained vision) ---
experiments["coral_vit_clip"] = {
    **base_config,
    "optim.loss_func": "coral",
    "model.timm_image.checkpoint_name": "vit_base_patch16_clip_224.laion2b_ft_in12k_in1k",
    "model.hf_text.checkpoint_name": "prajjwal1/bert-tiny",
}

print(f"Defined {len(experiments)} experiments:")
for name in experiments:
    cfg = experiments[name]
    loss = cfg.get("optim.loss_func", "cross_entropy")
    img = cfg.get("model.timm_image.checkpoint_name", "default")
    txt = cfg.get("model.hf_text.checkpoint_name", "N/A")
    names = cfg.get("model.names", "default (timm_image + hf_text + fusion_mlp)")
    print(f"  {name:40s} | loss={loss:15s} | img={img}")


Testing predictions on test data...


💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Output()

💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Output()


Predictions shape: (20,)
Predictions:
0    2
1    3
2    3
3    1
4    3
5    3
6    3
7    3
8    5
9    3
Name: rating, dtype: int64

Unique predicted values: [np.int64(1), np.int64(2), np.int64(3), np.int64(5)]

Testing predict_proba (probability outputs)...


Probabilities shape: (20, 5)

First 5 probability distributions:
          1         2         3         4         5
0  0.193754  0.223485  0.190285  0.209797  0.182679
1  0.183307  0.176777  0.254238  0.209016  0.176661
2  0.198294  0.166579  0.229737  0.211078  0.194311
3  0.228144  0.162556  0.203276  0.219948  0.186076
4  0.235119  0.161531  0.239180  0.166392  0.197777

Probabilities sum to 1? True

Comparing predictions with ground truth...
   True  Predicted
0     4          2
1     4          3
2     3          3
3     1          1
4     5          3
5     3          3
6     1          3
7     4          3
8     4          5
9     4          3

Test Accuracy: 25.00%
Mean Absolute Error: 1.300

CORAL LOSS TEST COMPLETED SUCCESSFULLY!


## Run Experiments

Select which experiments to run below. Results are collected in a DataFrame for comparison.

In [ ]:
# ============================================================
# SELECT EXPERIMENTS TO RUN
# ============================================================
# Comment/uncomment to choose which experiments to run.
# Start with a few fast ones, then add more.

experiments_to_run = [
    "coral_resnet18",                 # Fast baseline with CORAL
    "ce_resnet18",                    # CrossEntropy baseline (compare loss functions)
    "coral_resnet50",                 # Stronger CNN
    "coral_efficientnet_b0",          # Efficient architecture
    # "coral_convnext_tiny",          # Modern ConvNet (slower)
    # "coral_swin_base",             # Vision Transformer (slower, needs GPU)
    # "coral_resnet18_electra_small", # Better text encoder
    # "coral_resnet18_deberta_small", # Strong text encoder (slower)
    # "coral_resnet50_transformer_fusion",  # Transformer fusion
    # "coral_resnet18_ft_transformer",      # FT-Transformer for tabular
    # "coral_mobilenetv3",            # Lightweight
    # "coral_vit_clip",               # CLIP-pretrained ViT (needs GPU)
]

# ============================================================
# RUN LOOP
# ============================================================
results = []
predictors = {}

for exp_name in experiments_to_run:
    print(f"\n{'#'*70}")
    print(f"# EXPERIMENT: {exp_name}")
    print(f"{'#'*70}\n")

    config = experiments[exp_name]
    save_path = f"./experiment_models/{exp_name}"

    try:
        start_time = time.time()

        predictor = MultiModalPredictor(
            label=LABEL_COL,
            problem_type="multiclass",
            eval_metric="accuracy",
            path=save_path,
        )

        predictor.fit(
            train_data=train_df,
            hyperparameters=config,
            time_limit=TIME_LIMIT,
        )

        # Predict
        preds = predictor.predict(test_df.drop(columns=[LABEL_COL]))
        elapsed = time.time() - start_time

        # Compute metrics
        metrics = compute_ordinal_metrics(test_df[LABEL_COL].values, preds.values)
        metrics["experiment"] = exp_name
        metrics["loss"] = config.get("optim.loss_func", "cross_entropy")
        metrics["image_backbone"] = config.get("model.timm_image.checkpoint_name", "N/A")
        metrics["text_encoder"] = config.get("model.hf_text.checkpoint_name", "N/A")
        metrics["fusion"] = "transformer" if "fusion_transformer" in str(config.get("model.names", "")) else "mlp"
        metrics["time_sec"] = round(elapsed, 1)

        results.append(metrics)
        predictors[exp_name] = predictor

        print_metrics(metrics, name=exp_name)
        print(f"  Time: {elapsed:.1f}s")

    except Exception as e:
        print(f"\n❌ FAILED: {exp_name}")
        print(f"   Error: {e}")
        results.append({
            "experiment": exp_name,
            "accuracy": None,
            "within_one_accuracy": None,
            "mae": None,
            "quadratic_kappa": None,
            "loss": config.get("optim.loss_func", "cross_entropy"),
            "image_backbone": config.get("model.timm_image.checkpoint_name", "N/A"),
            "text_encoder": config.get("model.hf_text.checkpoint_name", "N/A"),
            "fusion": "N/A",
            "time_sec": None,
            "error": str(e),
        })

print(f"\n\n{'='*70}")
print(f"ALL EXPERIMENTS COMPLETED ({len(results)}/{len(experiments_to_run)})")
print(f"{'='*70}")

## Results Comparison

In [ ]:
# ============================================================
# RESULTS TABLE
# ============================================================
results_df = pd.DataFrame(results)

# Sort by within-one accuracy (most relevant for ordinal)
display_cols = [
    "experiment", "loss", "image_backbone", "text_encoder", "fusion",
    "accuracy", "within_one_accuracy", "mae", "quadratic_kappa", "time_sec"
]
available_cols = [c for c in display_cols if c in results_df.columns]
results_df_sorted = results_df[available_cols].sort_values("within_one_accuracy", ascending=False)

print("="*100)
print("EXPERIMENT RESULTS (sorted by Within-One Accuracy)")
print("="*100)
print(results_df_sorted.to_string(index=False, float_format="%.4f"))
print("="*100)

# Highlight best results
if results_df["within_one_accuracy"].notna().any():
    best_w1 = results_df.loc[results_df["within_one_accuracy"].idxmax()]
    best_acc = results_df.loc[results_df["accuracy"].idxmax()]
    best_mae = results_df.loc[results_df["mae"].idxmin()]
    best_qwk = results_df.loc[results_df["quadratic_kappa"].idxmax()]

    print(f"\n🏆 Best Within-One Accuracy: {best_w1['experiment']} ({best_w1['within_one_accuracy']:.4f})")
    print(f"🏆 Best Exact Accuracy:     {best_acc['experiment']} ({best_acc['accuracy']:.4f})")
    print(f"🏆 Best MAE:                {best_mae['experiment']} ({best_mae['mae']:.4f})")
    print(f"🏆 Best Quadratic Kappa:    {best_qwk['experiment']} ({best_qwk['quadratic_kappa']:.4f})")

# CORAL vs CrossEntropy comparison
coral_results = results_df[results_df["loss"] == "coral"]
ce_results = results_df[results_df["loss"] == "cross_entropy"]

if len(coral_results) > 0 and len(ce_results) > 0:
    print(f"\n{'='*60}")
    print("CORAL vs CrossEntropy (averaged across experiments)")
    print(f"{'='*60}")
    for metric in ["accuracy", "within_one_accuracy", "mae", "quadratic_kappa"]:
        coral_avg = coral_results[metric].mean()
        ce_avg = ce_results[metric].mean()
        better = "CORAL" if (coral_avg > ce_avg if metric != "mae" else coral_avg < ce_avg) else "CE"
        print(f"  {metric:25s}: CORAL={coral_avg:.4f}  CE={ce_avg:.4f}  → {better} wins")

## Detailed Analysis: Confusion Matrix & Error Distribution

In [ ]:
# ============================================================
# DETAILED ANALYSIS FOR BEST MODEL
# ============================================================

if results_df["within_one_accuracy"].notna().any():
    best_name = results_df.loc[results_df["within_one_accuracy"].idxmax(), "experiment"]
    best_pred = predictors[best_name]

    preds = best_pred.predict(test_df.drop(columns=[LABEL_COL]))
    y_true = test_df[LABEL_COL].values
    y_pred = preds.values

    # Confusion matrix
    classes = sorted(test_df[LABEL_COL].unique())
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    cm_df = pd.DataFrame(cm, index=[f"True {c}" for c in classes], columns=[f"Pred {c}" for c in classes])

    print(f"Confusion Matrix for: {best_name}")
    print(cm_df)

    # Error distribution
    errors = y_pred - y_true
    print(f"\nError Distribution (Predicted - True):")
    error_counts = pd.Series(errors).value_counts().sort_index()
    for err, count in error_counts.items():
        bar = "█" * int(count / max(error_counts) * 30)
        print(f"  {err:+d}: {count:4d} {bar}")

    # Per-class accuracy
    print(f"\nPer-Class Metrics:")
    for c in classes:
        mask = y_true == c
        if mask.sum() > 0:
            class_acc = (y_pred[mask] == c).mean()
            class_w1 = (np.abs(y_pred[mask] - c) <= 1).mean()
            class_mae = np.abs(y_pred[mask] - c).mean()
            print(f"  Class {c}: n={mask.sum():4d}  acc={class_acc:.3f}  within1={class_w1:.3f}  mae={class_mae:.3f}")

    # Probability analysis
    try:
        probs = best_pred.predict_proba(test_df.drop(columns=[LABEL_COL]))
        print(f"\nProbability Distribution (first 5 samples):")
        print(probs.head())
        print(f"\nProbabilities sum to 1? {np.allclose(probs.sum(axis=1), 1.0)}")
        print(f"Mean max probability (confidence): {probs.max(axis=1).mean():.4f}")
    except Exception as e:
        print(f"\nCould not compute probabilities: {e}")
else:
    print("No successful experiments to analyze.")

## Save Results

In [ ]:
# Save results to CSV
results_df.to_csv("experiment_results.csv", index=False)
print(f"Results saved to experiment_results.csv")
print(f"\nFinal summary:")
print(results_df_sorted[["experiment", "accuracy", "within_one_accuracy", "mae", "quadratic_kappa"]].to_string(index=False, float_format="%.4f"))